                        llama3-8b  pass@1: 0.38 | phi3 pass@1: 0.60  [HumanEval]

In [1]:
from data import CodeCompletionGenerator, load_config
from human_eval.data import write_jsonl, read_problems
from tqdm import tqdm as tqdm


config_name = "phi_3_instruct"
code_gen = CodeCompletionGenerator.from_config(config_name, device="mps")

# Get the error message on sub-batch and do fast adaptation 
system_prompt = "Do not fuck it up"
# Run Inference on HumanEval | Generate -> GetError -> SaveErro
# code_gen.run_human_eval_test(indices=indices, global_system_prompt=system_prompt)


Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

[WARNING] rope_scaling 'type' currently only supports 'linear' setting rope scaling to false.


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


#### Fast Adaptation

In [2]:
# Reflective Adaptation | Use Groq for fast iteration | CUDA device could also be fast (?)
from src.fast_adapt import *
from src.prompt import *
import numpy as np
# Initialize Sub-Batch Indices
indices = [55, 60]



# Key is to incorporate the information into node
class Node:
    def __init__(self, prompt, score, id, parent=None):
        self.prompt = prompt
        self.score = score
        self.id = id
        self.visits = 1
        self.parent = parent
        self.children = []

    def update_score(self, additional_score):
        self.score += additional_score
        self.visits += 1
        if self.parent:
            self.parent.update_score(additional_score) # Trace back to parent node -- add score along the trajectory


class MCTS:
    def __init__(self, root_prompt, id, infos):
        self.root = Node(prompt=root_prompt, score=base_score, id=id)
        self.nodes = [self.root]

    def select_node(self):
        # Select the node with the highest UCB1 score
        best_score = -1
        best_node = None
        for node in self.nodes:
            if node.visits == 0:
                return node
            score = node.score / node.visits + np.sqrt(2 * np.log(sum(n.visits for n in self.nodes)) / node.visits)
            if score > best_score:
                best_score = score
                best_node = node
        return best_node

    def expand_node(self, node, prompt, score, id): # Given a new prompt with verified score, expand new child node under current node
        new_node = Node(prompt=prompt, score=score, id=id, parent=node)
        node.children.append(new_node)
        self.nodes.append(new_node)

    def is_search_complete(self):
        # Define a condition to stop the search, e.g., a maximum number of nodes
        return len(self.nodes) > 100

    def get_best_prompt(self):
        # Return the prompt of the node with the highest average score
        best_node = max(self.nodes, key=lambda n: n.score / n.visits if n.visits > 0 else 0)
        return best_node.prompt

    def get_prompt_score(self, prompt):
        # Find the node with the given prompt and return its score
        for node in self.nodes:
            if node.prompt == prompt:
                return node.score / node.visits if node.visits > 0 else 0
        return 0

# Begin with a root node (prompt, score) pairs in short
init_prompt = "Do not fuck it up"
curr_id = 0

# code_gen.run_human_eval_test(indices=indices, global_system_prompt=init_prompt, id=curr_id)
infos = code_gen.get_info(indices, curr_id)
prompt_score = np.array([info['success'] for info in infos]).mean()

# Evolve 2 Child Nodes from Parent Node
case_infos = form_case_infos(infos)
prompts = call_rewriter(case_infos)



Call Function:  prompt_rewrite_tool {'error_analysis': 'The error encountered in the previous solution was related to an incorrect implementation of the Fibonacci function. The function did not return the correct Fibonacci number.', 'prompt': 'To fix the error, you should revise your implementation to correctly calculate the Fibonacci number.'}
Call Function:  prompt_rewrite_tool {'error_analysis': 'The function failed to pass all the unit tests.', 'prompt': 'Check your function implementation and make sure it correctly calculates the nth Fibonacci number.'}


['To fix the error, you should revise your implementation to correctly calculate the Fibonacci number.',
 'Check your function implementation and make sure it correctly calculates the nth Fibonacci number.']

In [11]:
root_node = Node.from_prompt(indices, init_prompt, curr_id, code_gen, parent=None)

Generating code completions:   0%|          | 0/2 [00:00<?, ?it/s]

Prompt: <s><|system|>
You are a friendly chatbot. [INSTRUCTION] Do not fuck it up [\INSTRUCTION] WRITE THE FULL COMPLETE FUNCTION (EG WITH def ....) END CODE WITH '```'. NOTE YOU ABSOLUTELY MUST END THE CODE WITH END CODE WITH '```' OR ELSE THE CODE WILL NOT BE INTERPRETTED!!!!<|end|>
<|user|>


def fib(n: int):
    """Return n-th Fibonacci number.
    >>> fib(10)
    55
    >>> fib(1)
    1
    >>> fib(8)
    21
    """
<|end|>
<|assistant|>


def fib(n: int):
    """Return n-th Fibonacci number.
    >>> fib(10)
    55
    >>> fib(1)
    1
    >>> fib(8)
    21
    """
    if n <= 0:
        return "Input should be a positive integer."
    elif n == 1:
        return 0
    elif n == 2:
        return 1
    else:
        a, b = 0, 1
        for _ in range(2, n):
            a, b = b, a + b
        return b

```<|end|><|assistant|> ```python

def fib(n: int):
    """Return n-th Fibonacci number.
    >>> fib(10)
    55
    >>> fib(1)
    1
    >>> fib(8)
    21
    """
    if n <= 0:
   

Generating code completions:  50%|█████     | 1/2 [00:12<00:12, 12.51s/it]

Prompt: 807.505 tokens-per-sec
Generation: 41.528 tokens-per-sec
Prompt: <s><|system|>
You are a friendly chatbot. [INSTRUCTION] Do not fuck it up [\INSTRUCTION] WRITE THE FULL COMPLETE FUNCTION (EG WITH def ....) END CODE WITH '```'. NOTE YOU ABSOLUTELY MUST END THE CODE WITH END CODE WITH '```' OR ELSE THE CODE WILL NOT BE INTERPRETTED!!!!<|end|>
<|user|>


def sum_to_n(n: int):
    """sum_to_n is a function that sums numbers from 1 to n.
    >>> sum_to_n(30)
    465
    >>> sum_to_n(100)
    5050
    >>> sum_to_n(5)
    15
    >>> sum_to_n(10)
    55
    >>> sum_to_n(1)
    1
    """
<|end|>
<|assistant|>


```python

def sum_to_n(n: int):
    """sum_to_n is a function that sums numbers from 1 to n.
    >>> sum_to_n(30)
    465
    >>> sum_to_n(100)
    5050
    >>> sum_to_n(5)
    15
    >>> sum_to_n(10)
    55
    >>> sum_to_n(1)
    1
    """
    return n * (n + 1) // 2
```<|end|><|assistant|> ```python

def sum_to_n(n: int):
    """sum_to_n is a function that sums numbers from 1

Generating code completions: 100%|██████████| 2/2 [00:21<00:00, 10.69s/it]


Prompt: 1204.431 tokens-per-sec
Generation: 41.667 tokens-per-sec
Writing error information to data/log/phi_3_instruct-55-60_0_info.jsonl...


100%|██████████| 2/2 [00:00<00:00, 29.94it/s]


In [12]:
child_nodes = root_node.spawn_child_nodes(indices, code_gen)

Call Function:  prompt_rewrite_tool {'error_analysis': 'The error encountered in the previous solution is related to the implementation of the Fibonacci sequence. The function does not return the correct Fibonacci numbers. The function should be revised to correctly calculate the Fibonacci sequence.', 'prompt': 'To avoid similar errors and reach the correct solution, consider revising the Fibonacci function to correctly calculate the sequence. You can use a loop to iterate from 2 to n and calculate the next Fibonacci number as the sum of the previous two numbers.'}
Call Function:  prompt_rewrite_tool {'error_analysis': 'The error message is due to incorrect implementation of the Fibonacci function. This could be due to incorrect initialization of variables or incorrect logic in the for loop.', 'prompt': 'To fix this error, make sure to correctly initialize the variables and use the correct logic in the for loop.'}


Generating code completions:   0%|          | 0/2 [00:00<?, ?it/s]

Prompt: <s><|system|>
You are a friendly chatbot. [INSTRUCTION] To avoid similar errors and reach the correct solution, consider revising the Fibonacci function to correctly calculate the sequence. You can use a loop to iterate from 2 to n and calculate the next Fibonacci number as the sum of the previous two numbers. [\INSTRUCTION] WRITE THE FULL COMPLETE FUNCTION (EG WITH def ....) END CODE WITH '```'. NOTE YOU ABSOLUTELY MUST END THE CODE WITH END CODE WITH '```' OR ELSE THE CODE WILL NOT BE INTERPRETTED!!!!<|end|>
<|user|>


def fib(n: int):
    """Return n-th Fibonacci number.
    >>> fib(10)
    55
    >>> fib(1)
    1
    >>> fib(8)
    21
    """
<|end|>
<|assistant|>


def fib(n: int):
    """Return n-th Fibonacci number.
    >>> fib(10)
    55
    >>> fib(1)
    1
    >>> fib(8)
    21
    """
    if n <= 0:
        return "Input should be a positive integer."
    elif n == 1:
        return 0
    elif n == 2:
        return 1
    else:
        a, b = 0, 1
        for _ in ra

Generating code completions:  50%|█████     | 1/2 [00:12<00:12, 12.74s/it]

Prompt: 627.937 tokens-per-sec
Generation: 41.215 tokens-per-sec
Prompt: <s><|system|>
You are a friendly chatbot. [INSTRUCTION] To avoid similar errors and reach the correct solution, consider revising the Fibonacci function to correctly calculate the sequence. You can use a loop to iterate from 2 to n and calculate the next Fibonacci number as the sum of the previous two numbers. [\INSTRUCTION] WRITE THE FULL COMPLETE FUNCTION (EG WITH def ....) END CODE WITH '```'. NOTE YOU ABSOLUTELY MUST END THE CODE WITH END CODE WITH '```' OR ELSE THE CODE WILL NOT BE INTERPRETTED!!!!<|end|>
<|user|>


def sum_to_n(n: int):
    """sum_to_n is a function that sums numbers from 1 to n.
    >>> sum_to_n(30)
    465
    >>> sum_to_n(100)
    5050
    >>> sum_to_n(5)
    15
    >>> sum_to_n(10)
    55
    >>> sum_to_n(1)
    1
    """
<|end|>
<|assistant|>


def sum_to_n(n: int):
    """
    This function calculates the sum of all numbers from 1 to n.
    """
    return n * (n + 1) // 2

```<|end|><|

...

Generating code completions:  50%|█████     | 1/2 [00:13<00:13, 13.15s/it]

Prompt: 1127.731 tokens-per-sec
Generation: 39.356 tokens-per-sec
Prompt: <s><|system|>
You are a friendly chatbot. [INSTRUCTION] To fix this error, make sure to correctly initialize the variables and use the correct logic in the for loop. [\INSTRUCTION] WRITE THE FULL COMPLETE FUNCTION (EG WITH def ....) END CODE WITH '```'. NOTE YOU ABSOLUTELY MUST END THE CODE WITH END CODE WITH '```' OR ELSE THE CODE WILL NOT BE INTERPRETTED!!!!<|end|>
<|user|>


def sum_to_n(n: int):
    """sum_to_n is a function that sums numbers from 1 to n.
    >>> sum_to_n(30)
    465
    >>> sum_to_n(100)
    5050
    >>> sum_to_n(5)
    15
    >>> sum_to_n(10)
    55
    >>> sum_to_n(1)
    1
    """
<|end|>
<|assistant|>


def sum_to_n(n: int):
    """
    This function calculates the sum of all numbers from 1 to n.
    """
    total_sum = 0
    for i in range(1, n+1):
        total_sum += i
    return total_sum

# Testing the function
print(sum_to_n(30))  # Expected output: 465
print(sum_to_n(100))  # Expe

Generating code completions: 100%|██████████| 2/2 [00:26<00:00, 13.47s/it]


Prompt: 815.782 tokens-per-sec
Generation: 37.880 tokens-per-sec
Writing error information to data/log/phi_3_instruct-55-60_1_info.jsonl...


100%|██████████| 2/2 [00:00<00:00, 18.67it/s]


In [ ]:
# Reflective Adaptation | Use Groq for fast iteration | CUDA device could also be fast (?)
from src.fast_adapt import *
from src.prompt import *
import numpy as np
# Initialize Sub-Batch Indices
indices = [55, 60]

prompt_infos = []
# Initialize System Prompt
curr_prompt = "Do not fuck it up"

curr_id = 0
# Load Runs | Check Prompt Score / Fitness
infos = code_gen.get_info(indices, curr_id)
prompt_info = {"prompt": system_prompt, "score": np.array([info['success'] for info in infos]).mean()}
prompt_infos.append({curr_id: prompt_info})
# Reflect & Rewrite
case_infos = form_case_infos(infos)
# Reflect & Rewrite Prompts
prompts = call_rewriter(case_infos)

for prompt in prompts:
    curr_id += 1
    # Check on new prompt's performance
    code_gen.run_human_eval_test(indices=indices, global_system_prompt=prompt, id=curr_id)
    infos = code_gen.get_info(indices, curr_id)
    prompt_info = {"prompt": system_prompt, "score": np.array([info['success'] for info in infos]).mean()}
    prompt_infos.append({curr_id: prompt_info})

    

In [ ]:
log_dir + filename + ".jsonl"

In [15]:
infos = code_gen.get_info(indices)


{'prompt': 'Do not fuck it up', 'score': 0.5}